# Traffic Flow Prediction using Custom GNN on PeMSD8


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Load `.npz` Data & Normalize

In [2]:
npz = np.load("/kaggle/input/pems-dataset/data/PEMS08/PEMS08.npz")
X = npz["data"]   # Shape: (T, N, F)
T, N, Fdim = X.shape

print("Shape of Dataset is",X.shape)


Shape of Dataset is (17856, 170, 3)


In [3]:

# Normalize each feature independently
scaler = StandardScaler()
X_reshaped = X.reshape(-1, Fdim)
X_scaled = scaler.fit_transform(X_reshaped).astype(np.float32)
X_norm = X_scaled.reshape(T, N, Fdim) # reshape to original dim

## Load Adjacency CSV & Build Edge Index

In [4]:
def load_adjacency(csv_path, num_nodes, threshold):
    df = pd.read_csv(csv_path)

    # torch.full creates a new tensor filled with a constant value.
    cost = torch.full((num_nodes, num_nodes), float('inf'))

    for _, row in df.iterrows():
        i, j, c = int(row['from']), int(row['to']), float(row['cost'])
        cost[i, j] = c

    cost.fill_diagonal_(0.0)  # Self-loops

    # Beacause we will only consider nearby neighbour for making adj matrix because it has greater influence 
    adj = (cost < threshold).float()

    edges = np.vstack(np.where(adj.numpy()))
    edge_index = torch.tensor(edges, dtype=torch.long)
    
    return adj.to(device), cost.to(device), edge_index.to(device)


In [5]:

NUM_NODES = 170   # Number of traffic sensors
THRESHOLD = 3000.0 # Distance threshold for connecting nodes in graph


adj, dist_matrix, edge_index = load_adjacency("/kaggle/input/pems-dataset/data/PEMS08/PEMS08.csv", NUM_NODES, THRESHOLD)
print("Edge index shape:", edge_index.shape)

Edge index shape: torch.Size([2, 446])


In [6]:

a = np.array([[5, 12, 7],
              [20, 3, 15]])

indices = np.where(a > 10)
print(indices)


(array([0, 1, 1]), array([1, 0, 2]))


## Generate Historical Sequences

In [7]:
TIME_LAGS = 12 # Input sequence length (12 timesteps)
HORIZON = 3  # Predict next 3 steps

def gen_sequences(Xarr, t=TIME_LAGS, h=HORIZON):
    XS, YS = [], []
    for i in range(len(Xarr) - t - h + 1):
        XS.append(Xarr[i:i+t])                  # Input: [t, N, F]
        YS.append(Xarr[i+t:i+t+h, :, 0])        # Output: [h, N] (flow only)
    return np.stack(XS), np.stack(YS)

X_seq, Y_seq = gen_sequences(X_norm)
print("X_seq shape:", X_seq.shape, "Y_seq shape:", Y_seq.shape)

X_seq shape: (17842, 12, 170, 3) Y_seq shape: (17842, 3, 170)


## Train/Val/Test Split + DataLoader

In [8]:
# 70% train 10% val and 20% test

train_t = int(0.7 * len(X_seq))
val_t = int(0.1 * len(X_seq))

X_train, Y_train = X_seq[:train_t], Y_seq[:train_t]
X_val,   Y_val   = X_seq[train_t:train_t+val_t], Y_seq[train_t:train_t+val_t]
X_test,  Y_test  = X_seq[train_t+val_t:], Y_seq[train_t+val_t:]

train_loader = DataLoader(TensorDataset(torch.tensor(X_train), torch.tensor(Y_train)), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.tensor(X_val), torch.tensor(Y_val)), batch_size=32, shuffle=False)
test_loader = DataLoader(TensorDataset(torch.tensor(X_test), torch.tensor(Y_test)), batch_size=32, shuffle=False)

## Define GNN Layer

In [9]:
class CustomGNNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super().__init__()
        self.W_self = nn.Linear(in_feats, out_feats, bias=False)
        self.W_neigh = nn.Linear(in_feats, out_feats, bias=False)

    def forward(self, X, edge_index):
        N, Feat = X.shape
        src, dst = edge_index

        h_self = self.W_self(X)  # shape: [N, out_feats]

        agg = torch.zeros_like(X)
        deg = torch.zeros(N, dtype=torch.float32, device=X.device)

        # Adding the neighbouring feature vector 
        for s, d in zip(src.tolist(), dst.tolist()):
            agg[d] += X[s]
            deg[d] += 1

        agg = agg / deg.clamp(min=1).unsqueeze(-1)
        
        h_neigh = self.W_neigh(agg)

        return F.relu(h_self + h_neigh)

## Build Model: Two GNN Layers + FC Output

In [10]:
class TrafficGNN(nn.Module):
    def __init__(self, feat_dim, hidden_dim, horizon):
        super().__init__()
        self.gnn1 = CustomGNNLayer(feat_dim, hidden_dim)
        self.gnn2 = CustomGNNLayer(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, horizon)

    def forward(self, X_batch, edge_index):
        B, t, N, F = X_batch.shape
        last = X_batch[:, -1]  # Take last timestep only
        out = []
        
        for b in range(B):
            x = last[b]  # [N, F]
            h = self.gnn1(x, edge_index)
            h = self.gnn2(h, edge_index)
            out.append(self.fc(h))  # [N, H]
            
        return torch.stack(out).permute(0, 2, 1)  # [B, H, N]


## Define Evaluation Function

In [11]:
flow_mean, flow_std = scaler.mean_[0], scaler.scale_[0]

def evaluate(loader):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.float().to(device), yb.float().to(device)
            y_pred = model(Xb, edge_index)

            y_pred = y_pred.cpu().numpy() * flow_std + flow_mean
            yb = yb.cpu().numpy() * flow_std + flow_mean

            preds.append(y_pred.reshape(-1))
            trues.append(yb.reshape(-1))

    preds, trues = np.concatenate(preds), np.concatenate(trues)
    return mean_absolute_error(trues, preds), np.sqrt(mean_squared_error(trues, preds)), r2_score(trues, preds)

## Train Model

In [ ]:

HIDDEN_DIM = 64           
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
EPOCHS = 30

model = TrafficGNN(Fdim, HIDDEN_DIM, HORIZON).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-3)
loss_fn = nn.MSELoss()


In [ ]:

best_val_rmse = float('inf')
print("Started Training...")

for epoch in range(1, 6):
    model.train()
    total_loss = 0.0

    for Xb, yb in train_loader:
        Xb, yb = Xb.float().to(device), yb.float().to(device)
        y_hat = model(Xb, edge_index)
        loss = loss_fn(y_hat, yb)

        opt.zero_grad()
        loss.backward()
        opt.step()

        total_loss += loss.item()

    val_mae, val_rmse, val_r2 = evaluate(val_loader)

    print(f"Epoch {epoch:2d} | Loss: {total_loss/len(train_loader):.4f} | Val MAE: {val_mae:.2f}, RMSE: {val_rmse:.2f}, R2: {val_r2:.3f}")

    if val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        torch.save(model.state_dict(), "best_traffic_gnn.pt")

## Final Test Results

In [ ]:
test_mae, test_rmse, test_r2 = evaluate(test_loader)
print(f"Test — MAE: {test_mae:.2f}, RMSE: {test_rmse:.2f}, R2: {test_r2:.3f}")

## Plot Prediction vs Actual for One Node

In [ ]:
model.eval()
all_preds, all_trues = [], []

with torch.no_grad():
    for Xb, yb in test_loader:
        Xb, yb = Xb.float().to(device), yb.float().to(device)
        y_hat = model(Xb, edge_index)

        y_hat = y_hat.cpu().numpy() * flow_std + flow_mean
        yb = yb.cpu().numpy() * flow_std + flow_mean

        all_preds.append(y_hat)
        all_trues.append(yb)

preds = np.concatenate(all_preds, axis=0)
trues = np.concatenate(all_trues, axis=0)

node_id = 0
h_step = 0

plt.figure(figsize=(12, 4))
plt.plot(trues[:100, h_step, node_id], label='Actual')
plt.plot(preds[:100, h_step, node_id], '--', label='Predicted')
plt.title(f'Traffic Flow at Node {node_id}, Horizon t+{h_step+1}')
plt.xlabel("Sample Index")
plt.ylabel("Flow")
plt.legend()
plt.grid(True)
plt.show()